Do a random forest return predictor for long time equity screens, maybe including rates/ sectors / commodities / geoploitic indicators. 
Find the % of regimes in the last 30 days that have a higher volatility than the current IV 
Add trend in shannon entropy or/and regime percentages to evaluate recessions / hostile environments beginning. could also look at earnings timestamps affecting options prices 

Want to find stocks with a low hv/iv high regime % in highest stdev state, and high # of days with stdev above avg iv on options 

In [4]:
import yfinance as yf 
from yfinance import EquityQuery
import numpy as np
import pandas as pd
import time


In [5]:
import numpy as np
import pandas as pd
import yfinance as yf
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from hmmlearn.hmm import GaussianHMM
from scipy.stats import entropy
from statsmodels.tsa.stattools import adfuller
from yfinance import EquityQuery, screen
from tqdm import tqdm

# ---------------- CONFIG ----------------
num_stocks = 250
max_workers = 6  # To reduce rate limit risk
hv_window = 20
price_volume_threshold = 100_000_000
batch_delay = 1.0  # seconds between batches
option_retry_delay = 5  # seconds retry delay for options fetch
N = 252  # annualization factor for volatility

# ---------------- HELPERS ----------------

def find_closest_value(data_list, target_value):
    if len(data_list) == 0:
        return None
    return min(data_list, key=lambda x: abs(x - target_value))

def filter_special_expiry(expiry_list):
    return sorted(expiry_list)[:2]

def shannon_entropy(series, bins=10):
    hist, _ = np.histogram(series, bins=bins, density=True)
    hist = hist[hist > 0]
    return entropy(hist, base=2)

def hmm_regime_switch_prob_and_count(returns, n_states=2):
    returns = returns.values.reshape(-1, 1)
    model = GaussianHMM(n_components=n_states, covariance_type="diag", n_iter=100)
    model.fit(returns)
    hidden_states = model.predict(returns)
    last_state = hidden_states[-1]
    transmat = model.transmat_
    switch_prob = 1 - transmat[last_state, last_state]
    regime_switch_count = np.sum(hidden_states[1:] != hidden_states[:-1])
    return switch_prob, regime_switch_count

def fit_hmm_and_stats(returns, n_states=3):
    model = GaussianHMM(
        n_components=n_states,
        covariance_type="diag",
        n_iter=500,
        tol=1e-2,
        random_state=42,
        verbose=False
    )
    X = returns.values.reshape(-1, 1)
    model.fit(X)
    hidden_states = model.predict(X)
    means = model.means_.flatten()
    stds = np.sqrt(np.array([np.diag(cov)[0] for cov in model.covars_]))
    return hidden_states, means, stds

def pct_regimes_higher_than_iv(hidden_states, stds, avg_iv):
    last30 = hidden_states[-30:]
    stds_annualized = stds * np.sqrt(N)
    last30_vols_annualized = stds_annualized[last30]
    pct_higher = np.mean(last30_vols_annualized > avg_iv) * 100
    return pct_higher

def hmm_analysis(returns, avg_iv):
    if returns is None or avg_iv is None:
        return {
            "hmm_mean_0": None, "hmm_std_0": None,
            "hmm_mean_1": None, "hmm_std_1": None,
            "hmm_mean_2": None, "hmm_std_2": None,
            "pct_last30_in_highest_stdev_state": None,
            "highest_stdev_state": None,
            "pct_regimes_last30_higher_than_avg_iv": None
        }
    hidden_states, means, stds = fit_hmm_and_stats(returns, n_states=3)
    high_stdev_state = int(np.argmax(stds))
    last30 = hidden_states[-30:]
    pct_high = np.mean(last30 == high_stdev_state) * 100
    pct_regimes_higher = pct_regimes_higher_than_iv(hidden_states, stds, avg_iv)
    return {
        "hmm_mean_0": means[0], "hmm_std_0": stds[0],
        "hmm_mean_1": means[1], "hmm_std_1": stds[1],
        "hmm_mean_2": means[2], "hmm_std_2": stds[2],
        "pct_last30_in_highest_stdev_state": pct_high,
        "highest_stdev_state": high_stdev_state,
        "pct_regimes_last30_higher_than_avg_iv": pct_regimes_higher
    }

def hmm_regime_minute_analysis(ticker):
    # Download last 7 days 1-min data
    try:
        df = yf.download(ticker, period="7d", interval="1m", progress=False)
        closes = df['Close'].dropna()
        if len(closes) < 1000:
            return np.nan, np.nan, np.nan, np.nan, np.nan
        log_returns = np.log(closes).diff().dropna()
        simple_returns = closes.pct_change().dropna()

        ent = shannon_entropy(log_returns)
        switch_prob, switch_count = hmm_regime_switch_prob_and_count(log_returns)
        adf_pvalue = adfuller(closes)[1]
        avg_return = simple_returns.mean()
        return ent, switch_prob*100, switch_count, adf_pvalue, avg_return
    except Exception:
        return np.nan, np.nan, np.nan, np.nan, np.nan

def get_options(symbol, current_price, max_retries=2, retry_delay=option_retry_delay):
    for attempt in range(max_retries):
        try:
            stock = yf.Ticker(symbol)
            options_chain = stock.options
            if not options_chain:
                return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])
            expiries = filter_special_expiry(options_chain)
            options_data = []
            if current_price is None:
                return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])
            for expiry in expiries:
                chain = stock.option_chain(expiry)
                for opt_type, df in [('call', chain.calls), ('put', chain.puts)]:
                    if not df.empty:
                        closest_strike = find_closest_value(df['strike'], current_price)
                        filtered = df[df['strike'] == closest_strike].copy()
                        filtered['type'] = opt_type
                        filtered['expiry'] = expiry
                        filtered['symbol'] = symbol
                        options_data.append(filtered)
            if options_data:
                result = pd.concat(options_data, ignore_index=True)
                result = result[result['openInterest'] > 500]
                return result[['strike', 'impliedVolatility', 'openInterest', 'symbol']]
            return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])
        except Exception:
            if attempt < max_retries - 1:
                time.sleep(retry_delay)
                continue
            else:
                return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])
    return pd.DataFrame(columns=['strike', 'impliedVolatility', 'openInterest', 'symbol'])

def fetch_symbol_data_all(symbol):
    try:
        ticker = yf.Ticker(symbol)
        hist = ticker.history(period="130d")  # 130d covers all needed windows + 7d intraday fetches separately handled
        closes = hist['Close'].dropna()
        volumes = hist['Volume'].dropna()
        if len(closes) < hv_window + 1 or len(closes) < 121 or len(volumes) < 2:
            return None
        # Previous day
        prev_day = hist.iloc[-2]
        price = float(prev_day['Close'])
        volume = int(prev_day['Volume'])
        if price * volume < price_volume_threshold:
            return None

        # Historical volatility (20 days)
        log_returns = np.log(closes / closes.shift(1)).dropna()
        hv = log_returns[-hv_window:].std() * np.sqrt(252)

        # Last 120 returns for HMM
        returns_120 = np.log(closes / closes.shift(1)).dropna()[-120:]
        if len(returns_120) < 120:
            returns_120 = None

        # Options data
        options_df = get_options(symbol, price)
        if options_df.empty:
            return None
        avg_iv = options_df['impliedVolatility'].mean()
        ratio = avg_iv / hv if hv > 0 else None
        mean_oi = options_df['openInterest'].mean()

        # Apply HMM for returns
        hmm_stats = hmm_analysis(returns_120, avg_iv)

        # 7-day 1m intraday regime analysis
        ent, switch_prob, switch_count, adf_pvalue, avg_return = hmm_regime_minute_analysis(symbol)

        return {
            'symbol': symbol.upper(),
            'price': price,
            'volume': volume,
            'historical_volatility': hv,
            'avg_iv': avg_iv,
            'iv/hv_ratio': ratio,
            'mean_openInterest': mean_oi,
            # HMM daily regimes stats
            **hmm_stats,
            # Intraday minute data stats
            'ShannonEntropy': ent,
            'Regimeswitchprob': switch_prob,
            'Regimeswitchcount': switch_count,
            'ADF_Pvalue': adf_pvalue,
            'AvgReturn_7d': avg_return,
            # Store returns for reference (optional, can be removed to reduce memory)
            #'returns_120': returns_120,
        }
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

# ---------------- YFINANCE SCREENER QUERY ----------------
screen_query = EquityQuery("and", [
    EquityQuery("is-in", ["region", "us"]),
    EquityQuery("is-in", ["sector",
        "Technology", "Financial Services", "Consumer Cyclical",
        "Communication Services", "Basic Materials", "Industrials"
    ]),
    EquityQuery("GTE", ["eodprice", 0.50]),
    EquityQuery("GTE", ["avgdailyvol3m", 100000]),
    EquityQuery("LTE", ["short_percentage_of_float.value", 100])
])

# ---------------- BATCH PROCESS ----------------
def process_batch(offset, filterdf, options):
    screener = screen(
        screen_query,
        sortField="avgdailyvol3m",
        sortAsc=False,
        size=num_stocks,
        offset=offset
    )
    stocks = screener.get('quotes', [])
    if not stocks:
        return filterdf, options, offset, False

    symbols = [s['symbol'] for s in stocks]
    results = []
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(fetch_symbol_data_all, sym): sym for sym in symbols}
        for future in as_completed(futures):
            data = future.result()
            if data:
                results.append(data)

    if not results:
        return filterdf, options, offset + num_stocks, True

    # Build DataFrame for symbol data (excluding options which is kept separate)
    batch_df = pd.DataFrame([{k:v for k,v in r.items() if k != "options_df"} for r in results])

    # Concatenate all options data into options DataFrame
    options_dfs = [r.get('options_df') for r in results if r.get('options_df') is not None]
    if options_dfs:
        all_options = pd.concat(options_dfs, ignore_index=True)
        options = pd.concat([options, all_options], ignore_index=True)

    filterdf = pd.concat([filterdf, batch_df], ignore_index=True)

    # Pause between batches to reduce rate limiting
    time.sleep(batch_delay)

    return filterdf, options, offset + num_stocks, True

# ---------------- MAIN ----------------
offset = 0
filterdf = pd.DataFrame()
options = pd.DataFrame()

while True:
    try:
        filterdf, options, offset, has_more = process_batch(offset, filterdf, options)
        print(f"Processed batch, total rows so far: {len(filterdf)}")
        if not has_more:
            break
    except Exception as e:
        if e.__class__.__name__ == "YFRateLimitError":
            print("Rate limit reached. Ending batch processing loop early.")
            break
        else:
            raise  # re-raise unknown exceptions



print("\nFinal results:")
print(filterdf)


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
Model is not converging.  Current: 14118.92868313863 is not greater than 14121.723691475716. Delta is -2.7950083370851644
Model is not converging.  Current: 14119.231218238974 is not greater than 14121.73907270127. Delta is -2.5078544622956542
Model is not converging.  Current: 14119.083992568101 is not greater than 14121.694030093648. Delta is -2.6100375255464314
Model is not converging.  Current: 14120.533357183856 is not greater than 14121.21657769436. Delta is -0.6832205105038156
Model is not converging.  Current: 14118.680847

Processed batch, total rows so far: 133


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
Model is not converging.  Current: 248.63020257530323 is not greater than 248.63291733598487. Delta is -0.002714760681641337
Model is not converging.  Current: 323.36532438568594 is not greater than 323.51399071866723. Delta is -0.14866633298129273
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
Model is not converging.  Current: 13248.958365662758 is not greater than 13250.404745703914. Delta is -1.4463800411558623
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  d

Processed batch, total rows so far: 211


Model is not converging.  Current: 303.42101419203556 is not greater than 303.65803839033595. Delta is -0.23702419830038934
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(

...

Processed batch, total rows so far: 273


Model is not converging.  Current: 293.6942808309349 is not greater than 293.7923925210471. Delta is -0.098111690112205
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
Model is not converging.  Current: 11291.668457665673 is not greater than 11293.105687570234. Delta is -1.4372299045608088
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
Model is not converging.  Current: 8814.32123127461 is not gr

Processed batch, total rows so far: 278


/var/folders/f_/1bh6mfq97bxdwzwnr49jf8nm0000gn/T/ipykernel_50345/1176544893.py:97: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period="7d", interval="1m", progress=False)
Model is not converging.  Current: 7648.103023576429 is not greater than 7648.704427866738. Delta is -0.6014042903088921


Processed batch, total rows so far: 279
Processed batch, total rows so far: 279
Processed batch, total rows so far: 279
Error fetching data for UI: Too Many Requests. Rate limited. Try after a while.
Error fetching data for UONEK: Too Many Requests. Rate limited. Try after a while.
Error fetching data for BALY: Too Many Requests. Rate limited. Try after a while.
Error fetching data for MUA: Too Many Requests. Rate limited. Try after a while.
Error fetching data for VRTS: Too Many Requests. Rate limited. Try after a while.
Error fetching data for YRD: Too Many Requests. Rate limited. Try after a while.
Error fetching data for COFS: Too Many Requests. Rate limited. Try after a while.
Error fetching data for BTT: Too Many Requests. Rate limited. Try after a while.
Error fetching data for HVT: Too Many Requests. Rate limited. Try after a while.
Error fetching data for QCRH: Too Many Requests. Rate limited. Try after a while.
Error fetching data for SNCR: Too Many Requests. Rate limited. Tr

In [6]:
# Show all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
filterdf["Shannon/IV"] = filterdf["ShannonEntropy"] / filterdf["avg_iv"]
display(filterdf)


,symbol,price,volume,historical_volatility,avg_iv,iv/hv_ratio,mean_openInterest,hmm_mean_0,hmm_std_0,hmm_mean_1,hmm_std_1,hmm_mean_2,hmm_std_2,pct_last30_in_highest_stdev_state,highest_stdev_state,pct_regimes_last30_higher_than_avg_iv,ShannonEntropy,Regimeswitchprob,Regimeswitchcount,ADF_Pvalue,AvgReturn_7d,Shannon/IV
0,PLTR,181.020004,53472900,0.431344,0.335212,0.777133,10985.000000,0.016163,0.028561,0.001052,0.042268,-0.033786,0.108194,0.000000,2,100.000000,1.224542,1.433604e-02,0.0,0.057119,Ticker NVDA -0.000001 dtype: float64,3.653039
1,F,11.440000,52469200,0.219591,0.241707,1.100714,12697.250000,0.001775,0.021355,0.003446,0.023175,-0.009879,0.063742,0.000000,2,100.000000,1.224542,1.390039e-02,0.0,0.057119,Ticker NVDA -0.000001 dtype: float64,5.066227
2,NVDA,182.020004,129554000,0.247410,0.310615,1.255466,35840.500000,0.009817,0.060145,-0.022765,0.042011,0.007424,0.021780,0.000000,0,100.000000,1.224542,1.467450e-02,0.0,0.057119,Ticker NVDA -0.000001 dtype: float64,3.942316
3,TSLA,335.579987,75000700,0.442691,0.270698,0.611484,9084.000000,0.000377,0.039570,0.003978,0.037661,-0.026392,0.118798,0.000000,2,100.000000,1.224542,1.823175e-02,0.0,0.057119,Ticker NVDA -0.000001 dtype: float64,4.523641
4,SOFI,23.290001,52370500,0.472104,0.309821,0.656257,4887.250000,0.012986,0.034474,0.001962,0.033463,-0.024042,0.090830,0.000000,2,100.000000,1.224542,1.327218e-02,0.0,0.057119,Ticker NVDA -0.000001 dtype: float64,3.952412
5,INTC,23.860001,188052500,0.561838,0.495122,0.881254,19366.250000,0.004966,0.030052,-0.007702,0.043408,0.126959,0.109599,0.000000,2,50.000000,1.224542,1.594031e-02,0.0,0.057119,Ticker NVDA -0.000001 dtype: float64,2.473211
6,AMD,180.949997,66308800,0.473977,0.280891,0.592626,11897.250000,0.007949,0.033387,-0.003158,0.034929,0.168632,0.123665,0.000000,2,100.000000,0.256628,1.823108e-01,10.0,0.884304,Ticker AAL 0.000043 dtype: float64,0.913622
7,APLD,14.550000,20751300,1.192506,0.595707,0.499542,2259.000000,-0.444996,0.100522,0.209798,0.114317,-0.004794,0.058884,3.333333,1,100.000000,0.256628,1.811576e-01,10.0,0.884304,Ticker AAL 0.000043 dtype: float64,0.430796
8,NIO,4.450000,43446300,0.805140,0.658207,0.817506,19078.500000,0.008070,0.041421,-0.012753,0.035139,0.095685,0.071445,6.666667,2,6.666667,0.256628,1.813345e-01,10.0,0.884304,Ticker AAL 0.000043 dtype: float64,0.389890
9,AAPL,232.779999,51916300,0.289689,0.170571,0.588807,9604.750000,-0.050356,0.062713,0.010414,0.100058,0.000481,0.019666,0.000000,1,100.000000,0.256628,1.822639e-01,10.0,0.884304,Ticker AAL 0.000043 dtype: float64,1.504524


In [8]:
import datetime
filterdf["AvgReturn_7d"] = 0 
today = datetime.datetime.today().strftime('%Y-%m-%d')
filename = f'/Users/nshaffer/Desktop/equity_vol_screen_{today}.csv'
filterdf.to_csv(filename, index=False)
print(f"Results saved to {filename}")

Results saved to /Users/nshaffer/Desktop/equity_vol_screen_2025-08-15.csv


In [7]:
import nest_asyncio
nest_asyncio.apply()
import asyncio
from ib_insync import *
import pandas as pd
import numpy as np
import ta

# ===== CONFIG =====
BAR_SIZE = '1 day'
DURATION = '5 Y'
NUM_BARS = 2000
VOL_WINDOW = 30
VOL_THRESHOLD = 0.001
REGIME_THRESHOLD = -0.1
ADX_PERIOD = 14
ADX_THRESHOLD = 20
SMA_PERIOD = 200
EXCHANGE = 'SMART'
CURRENCY = 'USD'
LOOKAHEAD = 4
K_NEIGHBORS = 8
FEATURES = ['RSI', 'ADX', 'CCI', 'MACD']


# Preload IBKR connection for reuse
ib = IB()

async def fetch_ibkr_stock(symbol):
    contract = Stock(symbol, EXCHANGE, CURRENCY)
    await ib.qualifyContractsAsync(contract)
    bars = await ib.reqHistoricalDataAsync(
        contract=contract,
        endDateTime='',
        durationStr=DURATION,
        barSizeSetting=BAR_SIZE,
        whatToShow='TRADES',
        useRTH=True,
        formatDate=1
    )
    df = util.df(bars).set_index('date').tail(NUM_BARS)
    return symbol, df


def lorentzian_distance_matrix(target, anchors):
    """Vectorized Lorentzian distance calculation."""
    return np.sum(np.log1p(np.abs(anchors - target)), axis=1)


def volatility_filter(df):
    atr = ta.volatility.AverageTrueRange(
        df['high'], df['low'], df['close'], window=VOL_WINDOW
    ).average_true_range()
    return (atr / df['close']).to_numpy() > VOL_THRESHOLD


def regime_filter(df):
    returns = df['close'].pct_change()
    return returns.rolling(window=20).mean().to_numpy() > REGIME_THRESHOLD


def adx_filter(df):
    adx = ta.trend.ADXIndicator(
        df['high'], df['low'], df['close'], window=ADX_PERIOD
    ).adx()
    return adx.to_numpy() > ADX_THRESHOLD


def classify_lorentzian_knn_with_filters(df):
    closes = df['close'].values
    feature_mat = df[FEATURES].values
    sma = df['SMA'].values

    vol_mask = volatility_filter(df)
    regime_mask = regime_filter(df)
    adx_mask = adx_filter(df)

    valid_filter = vol_mask & regime_mask & adx_mask

    pred = np.full(len(df), np.nan)
    length = len(df)

    for idx in range(500, length):
        if not valid_filter[idx] or np.isnan(sma[idx]) or closes[idx] == 0:
            continue

        anchor_start = max(0, idx - 500)
        anchors = feature_mat[anchor_start:idx]
        if anchors.shape[0] < K_NEIGHBORS:
            continue

        dists = lorentzian_distance_matrix(feature_mat[idx], anchors)
        knn_indices = np.argsort(dists)[:K_NEIGHBORS]
        anchor_indices = np.arange(anchor_start, idx)[knn_indices]

        valid_idx = anchor_indices + LOOKAHEAD < length
        selected = anchor_indices[valid_idx]
        if len(selected) == 0:
            continue

        y_train = np.sign(closes[selected + LOOKAHEAD] - closes[selected])
        vote = y_train.sum()
        raw_signal = np.sign(vote) if vote != 0 else 0

        if raw_signal == 1 and closes[idx] > sma[idx]:
            pred[idx] = 1
        elif raw_signal == -1 and closes[idx] < sma[idx]:
            pred[idx] = -1
        else:
            pred[idx] = 0

    return pred


async def process_symbols_with_knn(filterdf):
    await ib.connectAsync('127.0.0.1', 7497, clientId=np.random.randint(1000, 10000))
    
    # Fetch all data concurrently
    tasks = [fetch_ibkr_stock(sym) for sym in filterdf['symbol']]
    results = await asyncio.gather(*tasks)

    for symbol, df in results:
        print(f"Processing {symbol}...")
        df['RSI'] = ta.momentum.RSIIndicator(df['close'], window=14).rsi()
        df['ADX'] = ta.trend.ADXIndicator(df['high'], df['low'], df['close'], window=14).adx()
        df['CCI'] = ta.trend.CCIIndicator(df['high'], df['low'], df['close'], window=20).cci()
        df['SMA'] = ta.trend.SMAIndicator(df['close'], window=SMA_PERIOD).sma_indicator()
        macd_ind = ta.trend.MACD(df['close'])
        df['MACD'] = macd_ind.macd()

        df = df.dropna().reset_index(drop=True)

        preds = classify_lorentzian_knn_with_filters(df)
        last_8 = preds[-8:]
        
        pct_buy = np.sum(last_8 == 1) / len(last_8) * 100
        pct_sell = np.sum(last_8 == -1) / len(last_8) * 100

        filterdf.loc[filterdf['symbol'] == symbol, 'pct_buy_last_8d'] = pct_buy
        filterdf.loc[filterdf['symbol'] == symbol, 'pct_sell_last_8d'] = pct_sell

    ib.disconnect()
    return filterdf


if __name__ == '__main__':
    # Example dataframe

    df_results = asyncio.run(process_symbols_with_knn(filterdf))
    print(df_results)


API connection failed: ConnectionRefusedError(61, "Connect call failed ('127.0.0.1', 7497)")
Make sure API port on TWS/IBG is open


ConnectionRefusedError: [Errno 61] Connect call failed ('127.0.0.1', 7497)

In [ ]:
filterdf

,language,region,quoteType,typeDisp,quoteSourceName,triggerable,customPriceAlertConfidence,currency,shortName,bid,ask,regularMarketChangePercent,exchange,fiftyTwoWeekHigh,fiftyTwoWeekLow,averageAnalystRating,dividendYield,hasPrePostMarketData,firstTradeDateMilliseconds,priceHint,postMarketChangePercent,postMarketTime,postMarketPrice,postMarketChange,regularMarketChange,regularMarketTime,regularMarketPrice,regularMarketDayHigh,regularMarketDayRange,regularMarketDayLow,regularMarketVolume,regularMarketPreviousClose,bidSize,askSize,market,messageBoardId,fullExchangeName,longName,financialCurrency,regularMarketOpen,averageDailyVolume3Month,averageDailyVolume10Day,corporateActions,fiftyTwoWeekLowChange,fiftyTwoWeekLowChangePercent,fiftyTwoWeekRange,fiftyTwoWeekHighChange,fiftyTwoWeekHighChangePercent,fiftyTwoWeekChangePercent,dividendDate,earningsTimestamp,earningsTimestampStart,earningsTimestampEnd,earningsCallTimestampStart,earningsCallTimestampEnd,isEarningsDateEstimate,trailingAnnualDividendRate,trailingPE,dividendRate,trailingAnnualDividendYield,marketState,epsTrailingTwelveMonths,epsForward,epsCurrentYear,priceEpsCurrentYear,sharesOutstanding,bookValue,fiftyDayAverage,fiftyDayAverageChange,fiftyDayAverageChangePercent,twoHundredDayAverage,twoHundredDayAverageChange,twoHundredDayAverageChangePercent,marketCap,forwardPE,priceToBook,sourceInterval,exchangeDataDelayedBy,exchangeTimezoneName,exchangeTimezoneShortName,gmtOffSetMilliseconds,esgPopulated,tradeable,cryptoTradeable,displayName,symbol,ipoExpectedDate,prev_day_price,prev_day_volume,price_volume,historical_volatility,avg_iv,iv/hv_ratio,mean_openInterest_x,mean_openInterest_y,mean_openInterest,hmm_mean_0,hmm_std_0,hmm_mean_1,hmm_std_1,hmm_mean_2,hmm_std_2,pct_last30_in_highest_stdev_state,highest_stdev_state,pct_regimes_last30_higher_than_avg_iv,ShannonEntropy,Regimeswitchprob,Regimeswitchcount,ADF_Pvalue,AvgReturn_7d,Shannon/IV,pct_buy_last_8d,pct_sell_last_8d
0,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,NVIDIA Corporation,173.61,182.83,-2.333160,NMS,183.300000,86.620000,1.4 - Strong Buy,0.02,True,917015400000,2,-0.702280,1.754093e+09,172.5000,-1.220000,-4.149990,1754078401,173.720,176.5400,170.89 - 176.54,170.8900,202636888,177.870,4,1,us_market,finmb_32307,NasdaqGS,NVIDIA Corporation,USD,174.0900,192781287,161557370,[],87.100000,1.005541,86.62 - 183.3,-9.580002,-0.052264,72.941760,1.751501e+09,1.756325e+09,1.756325e+09,1.756325e+09,1.756328e+09,1.756328e+09,False,0.040,56.220066,0.04,0.000225,CLOSED,3.09,4.12,4.30288,40.372963,24387600384,3.438,154.51160,19.208405,0.124317,134.764800,38.955200,0.289061,4236614041600,42.165050,50.529377,15,0,America/New_York,EDT,-14400000,False,False,False,NVIDIA,NVDA,NaN,177.869995,221685400,39431181015.551758,0.253826,0.394598,1.5546,9096.000000,9096.000000,9096.0,0.006675,0.031073,-0.004445,0.032405,0.110727,0.120742,0.0,2,100.0,0.833454,0.10,6,0.218884,Ticker NVDA 0.000003 dtype: float64,2.112159,100.0,0.0
1,en-US,US,EQUITY,Equity,Nasdaq Real Time Price,True,HIGH,USD,"Lucid Group, Inc.",2.40,2.47,-1.626010,NMS,4.430000,1.930000,3.1 - Hold,NaN,True,1600435800000,4,0.206606,1.754093e+09,2.4250,0.005000,-0.040000,1754078401,2.420,2.4400,2.32 - 2.44,2.3200,91282600,2.460,488,575,us_market,finmb_83747444,NasdaqGS,"Lucid Group, Inc.",USD,2.3750,152561575,131673640,[],0.490000,0.253886,1.93 - 4.43,-2.010000,-0.453725,-19.333332,NaN,1.754424e+09,1.754424e+09,1.754424e+09,1.754429e+09,1.754429e+09,False,0.000,NaN,NaN,0.000000,CLOSED,-1.19,-0.88,-0.88780,-2.725839,3072480000,1.044,2.39080,0.029200,0.012214,2.516975,-0.096975,-0.038528,7435401728,-2.750000,2.318008,15,0,America/New_York,EDT,-14400000,False,False,False,Lucid,LCID,2021-07-26,2.46,65248900,160512296.489048,1.334923,1.160160,0.869084,28425.250000,28425.250000,28425.25,0.003372,0.051561,-0.011354,0.040461,0.30257,0.106913,3.333333,2,3.333333,1.378179,0.14,7,0.780178,Ticker LCID -0.000083 dtype: float64,1.187921,25.0,25.0
2,en-US,US,EQUITY,Equity,Nasdaq Real Time Pri

In [ ]:
import datetime
filterdf["AvgReturn_7d"] = 0 
today = datetime.datetime.today().strftime('%Y-%m-%d')
filename = f'/Users/nshaffer/Desktop/equity_vol_screen_{today}.csv'
filterdf.to_csv(filename, index=False)
print(f"Results saved to {filename}")


Results saved to /Users/nshaffer/Desktop/equity_vol_screen_2025-08-02.csv
